This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 325.49788774, ...,   0.        ,
        23.809066  ,   0.        ]), array([8.86415225e+01, 7.51855248e+01, 1.62748964e+02, ...,
       1.05541276e-62, 1.19045330e+01, 0.00000000e+00]), OrderedDict([('R2', 0.9972001170883225), ('root_mean_squared_error', 71.18352103539858), ('mean_absolute_error', 52.88845046529206), ('rmse_over_stdev', 0.7452809118336613)]), None)
test_quad = (array([2.02092457e+02, 2.45647989e+02, 2.19470531e+02, 2.09634413e+02,
       3.40489474e+02, 3.88395179e+02, 2.59721126e+02, 1.40876460e+02,
       1.10462825e+02, 9.45736257e+01, 9.62792101e+01, 9.53431111e+01,
       2.41991216e+02, 1.45936466e+02, 3.04216098e+02, 1.23826343e+02,
       1.77346326e+02, 8.94641218e+01, 0.00000000e+00, 3.21795003e+02,
       1.70828354e+02, 1.95159840e+02, 1.96095876e+02, 1.37316410e+02,
       3.50019597e+02, 3.02985863e+02, 3.19706165e+02, 2.27914069e+02,
       8.44124962e+01, 1.03964994e+02, 2.09882629e+02, 7.02166732e+01,
       2.16105345e+01, 2.89476692e+01, 6.30457358e+01, 3.87490899e+01,
       2.56639133e+01, 7.66050637e+01, 1.18303763e+02, 4.17751932e+02,
       1.29128414e+02, 1.00971120e+02, 6.36339325e+02, 6.86792014e+01,
       2.72804393e+02, 1.78235739e+02, 2.27491285e+02, 1.22311356e+02,
       2.74349274e+02, 2.78307782e+02, 1.82740912e+02, 0.00000000e+00,
       9.04068336e+01, 7.58782445e+01, 6.92869614e+01, 1.29458568e+01,
       5.42920320e+01, 3.57160729e+02, 4.00299318e+01, 2.68896498e+01,
       0.00000000e+00, 7.80236106e+01, 1.16763996e+00, 0.00000000e+00,
       0.00000000e+00, 6.61459738e+01, 8.03088872e+01, 1.77441804e+02,
       9.68586984e+01, 7.03541024e+01, 1.27239967e+02, 2.53560446e+02,
       2.51771744e+02, 5.95521104e+01, 4.11709055e+01, 4.52159618e+01,
       0.00000000e+00, 1.66684097e+01, 0.00000000e+00, 2.03972667e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.39741468e+01, 3.20782685e+01, 1.54429064e+01, 1.73373103e+01,
       7.53897668e+01, 0.00000000e+00, 3.49408401e+01, 6.03605982e+01,
       3.27899307e+01, 6.05219124e+01, 2.67022126e+01, 0.00000000e+00,
       4.42073704e-01, 1.78001455e+02, 1.81910775e+02, 1.31304680e+02,
       1.14247314e+02, 9.92490625e-01, 4.93632300e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.48100516e+01,
       4.23248061e+01, 0.00000000e+00, 2.57222520e+01, 2.21842737e+01,
       2.08989075e+02, 5.38520339e+01, 3.13780640e+01, 3.66852424e+01,
       3.51232216e+01, 3.51583952e+01, 7.30016409e+01, 1.05693165e+02,
       7.80094958e+01, 1.32109024e+02, 7.79804428e+01, 1.23754016e+02,
       2.68325976e+01, 0.00000000e+00, 2.78984665e+02, 2.24784003e+01,
       5.91838239e+01, 1.25292208e+02, 6.99723855e+01, 4.70862505e+01,
       9.34258071e+01, 1.14554644e+02, 1.58427043e+02, 1.09033411e+02,
       2.57075963e+02, 2.63839575e+02, 2.76340269e+02, 8.39171957e+01,
       0.00000000e+00, 0.00000000e+00, 2.80876470e+02, 2.11460716e+02,
       2.81102443e+02, 2.86014919e+02, 2.02283948e+02, 2.30921905e+02,
       1.28448866e+02, 5.08392287e+01, 2.91802070e+01, 1.31808017e+01,
       0.00000000e+00, 0.00000000e+00, 2.93008151e+01, 0.00000000e+00,
       0.00000000e+00, 1.21655324e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.84401941e+01, 5.12388154e+01,
       3.08078053e+01, 4.81001678e+01, 4.45494170e+01, 2.34800709e+02,
       7.54658121e+01, 8.29266298e+01, 6.06316356e+01, 4.63304068e+01,
       1.10052006e+02, 9.70888906e+01, 5.13090164e+01, 5.33902792e+01,
       4.43244216e+01, 0.00000000e+00, 3.92079232e+01, 1.54697467e+02,
       1.02151271e+02, 8.15909270e+01, 9.38711282e+01, 0.00000000e+00,
       2.64912691e+01, 4.00673869e+01, 4.95815034e+01, 6.33172566e+01,
       1.02395743e+02, 3.11645320e+01, 1.76733958e+01, 0.00000000e+00,
       2.75959946e+01, 1.40515010e+02, 4.27492462e+01, 5.19507218e+01,
       1.01745118e+01, 1.92994335e+01, 0.00000000e+00, 9.40584552e+01,
       1.05442368e+02, 6.89012756e+00, 2.32118071e+01, 6.90074178e+01,
       0.00000000e+00, 2.61974495e+01, 0.00000000e+00, 3.56929084e+01,
       1.20413789e+02, 1.55663046e+02, 1.93229456e+02, 1.83828681e+02,
       3.42040019e+02, 3.66503081e+02, 2.76006928e+02, 9.15340044e+01,
       1.13869162e+02, 1.18103266e+02, 2.20003381e+02, 1.39013581e+02,
       2.59886084e+02, 2.85344428e+02, 1.44708713e+02, 1.44006558e+02,
       1.90075255e+02, 8.30462436e+01, 1.21428977e+02, 2.31481855e+02,
       1.77752168e+02, 2.51196620e+02, 1.54419624e+02, 2.36338029e+02,
       2.50579269e+02, 2.67340580e+02, 2.62896301e+02, 2.21055904e+02,
       3.40306363e+02, 1.89648365e+02, 2.33810236e+02, 3.37214376e+02,
       1.87894409e+02, 5.98511381e+01, 1.80660958e+02, 1.10209088e+02,
       9.69915119e+01, 1.49647778e+02, 1.93969856e+02, 1.00763646e+02,
       1.88513164e+02, 1.31977595e+02, 1.02794584e+02, 1.21127399e+02,
       2.53019864e+02, 3.32212137e+02, 2.52001050e+02, 3.17356871e+02,
       2.98525793e+02, 3.23634151e+02, 2.23307827e+02, 3.18434650e+02,
       8.54936441e+01, 7.99859482e+01, 1.34599602e+02, 1.48873887e+02,
       7.93248059e+01, 1.09381016e+02, 6.02850907e+01, 5.55045542e+01,
       1.50801863e+02, 1.60434807e+02, 1.80035168e+02, 2.72871580e+02,
       2.02525534e+02, 2.10329787e+02, 2.99112578e+02, 1.44135045e+02,
       1.73746536e+02, 1.83572992e+02, 2.04542101e+02, 2.71906598e+02,
       1.51155130e+02, 1.10472455e+02, 1.35426489e+02, 9.88049065e+01,
       1.07655170e+02, 8.10050964e+01, 9.27064680e+01, 9.69622803e+01,
       1.31798616e+02, 1.05836151e+02, 1.08312182e+02, 1.89420372e+02,
       1.22185191e+02, 1.02217058e+02, 1.10480194e+02, 6.28478449e+01,
       4.68826690e+01, 2.72145096e+01, 9.31208466e+01, 4.43281812e+01,
       6.20242803e+01, 4.34380611e+01, 1.25171931e+02, 1.82785945e+01,
       2.20249128e+01, 5.66837962e+01, 7.58528445e+01, 1.31671414e+02,
       5.12762345e+01, 1.40218828e+02, 1.78369049e+02, 1.84215977e+02,
       1.13527293e+02, 1.85414463e+02, 1.63793911e+02, 1.42933388e+02,
       1.84787732e+02, 1.59787224e+02, 1.47483468e+02, 1.63982208e+02,
       2.98870605e+01, 1.05722180e+02, 1.09488956e+02, 9.60360384e+01,
       1.26798490e+02, 1.04372971e+02, 1.04244208e+02, 1.88514713e+02,
       1.85051275e+02, 1.15594393e+02, 1.09297300e+02, 1.62419081e+02,
       1.37369485e+02, 1.91314596e+02, 6.97752946e+01, 2.97477073e+01,
       3.71426186e+01, 3.63667394e+01, 3.48221974e+01, 1.97610067e+02,
       2.15562064e+01, 7.10907636e+01, 8.71299483e+01, 1.91667256e+01,
       5.97413320e+01, 3.97988019e+01, 5.91429461e+01, 1.98665047e+02,
       1.34244899e+02, 3.52832986e+01, 9.38114347e+01, 9.98669764e+01,
       1.43739984e+02, 3.81360533e+01, 1.23689710e+02, 0.00000000e+00,
       2.41527621e+01, 0.00000000e+00, 0.00000000e+00, 5.32856309e+01,
       4.94793928e+01, 7.26725204e+01, 1.00472922e+02, 7.32347672e+01,
       3.97951736e+01, 0.00000000e+00, 4.87758689e+01, 1.60849161e+02,
       4.34643306e+01, 6.39648556e+01, 2.12813897e+01, 2.63539598e+01,
       3.38853640e+01, 2.35812529e+01, 3.73769471e+01, 7.48181406e+01,
       0.00000000e+00, 3.40153297e+01, 2.67854426e+02, 2.07757183e+02,
       8.29965720e+01, 7.72377141e+01, 1.52154945e+02, 6.98177552e+01,
       5.51408019e+01, 6.05410832e+01, 8.22454137e+01, 6.94738840e+01,
       4.54995709e+01, 4.38219567e+01, 4.23854981e+01, 3.83815528e+01,
       0.00000000e+00, 0.00000000e+00, 1.77019953e+01, 7.17619802e+01,
       3.15229082e+01, 0.00000000e+00, 9.46025620e+01, 8.48759224e+01,
       1.42323884e+02, 3.74352793e+01, 3.12618672e+01, 2.59562148e+01,
       8.01561257e+01, 5.19044272e+01, 3.87785474e+01, 5.43341092e+01,
       1.99708105e+01, 6.57797960e+01, 2.51156351e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.88957798e+01,
       4.72079029e+01]), array([ 2.72701871e-005,  3.02068169e-143,  0.00000000e+000,
        0.00000000e+000,  9.26783892e-123,  0.00000000e+000,
        0.00000000e+000,  3.28018776e-133,  6.13395499e-192,
        2.43976360e-028,  3.97993955e-192,  0.00000000e+000,
        7.09097464e-028,  1.04504870e-115,  3.03592254e-115,
        4.89840557e-192,  1.75877285e-132,  0.00000000e+000,
        5.22041378e-308,  3.72703435e-010,  2.87097308e-130,
        8.39958755e-236,  2.18508045e-130,  0.00000000e+000,
        5.76033302e-236,  9.12026138e-123,  4.78919061e-236,
        0.00000000e+000,  2.97105963e-032,  1.77238648e-252,
        1.74453638e-274,  1.05391046e-008,  3.11094274e-119,
        1.97450682e-012,  3.16813674e+001,  4.20480917e-014,
        4.27152655e-038,  1.59648651e-038,  2.82504780e-001,
        8.60538049e-137,  3.01973614e-028,  2.43986338e-004,
        0.00000000e+000,  2.13492880e-021,  0.00000000e+000,
        1.63420230e-092,  0.00000000e+000,  6.11556778e+001,
        0.00000000e+000,  0.00000000e+000,  2.35617013e-025,
        0.00000000e+000,  0.00000000e+000,  1.94637402e-223,
        5.61906052e-099,  1.52778747e-026,  3.59148000e-039,
        0.00000000e+000,  4.21271129e-002,  1.64059738e-010,
       -2.99682182e-047,  5.88984428e-005,  0.00000000e+000,
        1.75979937e-252,  3.33909796e-112,  7.62302838e-006,
        0.00000000e+000,  1.58846514e-108,  1.87882477e-011,
        3.88474556e-278,  4.91228408e-032,  4.64666983e-002,
        1.30305277e-025,  2.67121120e-010,  5.82975368e-001,
        2.43741893e-072,  0.00000000e+000,  2.56891819e-084,
        6.55852253e-241,  4.47601138e-238,  0.00000000e+000,
       -6.97627038e-250,  9.92751473e-239, -4.84159029e-241,
       -9.09996770e-017,  2.35848556e-031,  0.00000000e+000,
       -3.64202830e-027,  1.48987562e-085,  0.00000000e+000,
        1.96247945e-236,  5.89732765e-239,  5.31329223e-006,
        2.72481239e+000,  8.75206505e-011,  0.00000000e+000,
        1.06255410e-094,  9.57739162e-011,  2.16852008e-001,
        1.27671677e-006,  9.96131753e-072, -2.89590717e-249,
        4.86048175e-076,  3.09297618e-033,  2.89952417e-095,
        2.03445047e-010,  8.61992794e-009,  0.00000000e+000,
        1.66655712e-004,  1.76943421e-094,  5.93969568e-010,
        2.29517165e+000,  5.96393340e-005,  0.00000000e+000,
        8.79447528e-073, -5.20717587e-254,  3.64212853e-255,
        2.34287156e-242,  0.00000000e+000,  5.69943804e-085,
        2.90501245e-006,  5.43153260e-073,  7.68883791e-013,
        1.61027419e-052,  6.89642891e-007,  3.29453411e-084,
        2.01693177e-013,  3.20478793e+000,  3.56395336e-030,
        3.55388763e-013,  6.28177037e-006,  0.00000000e+000,
        1.77377797e-084,  4.55868298e-205,  0.00000000e+000,
        2.92103185e+000,  0.00000000e+000,  1.00197619e-048,
        1.78894259e-048,  1.18422900e-110,  1.54055481e-048,
        1.01368092e-022,  1.11941138e-286,  9.42535745e-014,
        1.01702160e-022,  3.13941910e-011,  4.53303220e-005,
        8.52268226e-023,  5.38206015e-011,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        2.59556479e+001,  1.54938040e-028,  4.96709567e-038,
        6.05063252e-083,  2.56940117e-079,  1.20457103e-001,
        2.01308150e-047,  6.09009675e-005,  1.64680306e-006,
        7.88481282e-002,  4.80399862e-019,  0.00000000e+000,
        0.00000000e+000,  5.04692552e-002,  0.00000000e+000,
        3.52868591e-001,  5.56873917e-077,  1.78482924e-053,
        2.02905172e-031, -3.33018493e-017, -8.66804828e-100,
        9.97111355e-023,  1.76032714e-006,  3.05088379e-003,
        5.13456360e+000,  7.69096438e-183,  1.21240725e-087,
        3.08701984e-006,  1.16595439e-003,  1.20633822e-006,
        4.08519860e-006,  1.13648580e-007,  3.72937803e-012,
        4.15167873e-088,  7.86802827e-008,  4.15434115e+000,
        0.00000000e+000,  1.23612033e-253,  1.82551624e-007,
        2.13542873e-020,  1.54819941e-008,  3.07792539e+000,
        0.00000000e+000,  1.97856615e-002,  4.96404166e-133,
        2.37022575e-133,  0.00000000e+000,  0.00000000e+000,
        2.54432688e-017,  1.69963947e-009,  6.02276300e-038,
        4.10574879e-017,  4.91208784e-034,  4.68203817e-017,
        1.67359879e-157,  9.01529437e-089,  3.73640801e-102,
        0.00000000e+000,  6.72490932e-009,  1.74874865e-004,
        3.83692631e-134,  9.31474864e-001,  1.78345258e-013,
        4.75926394e-199,  6.55603988e-014,  0.00000000e+000,
        4.79411066e-004,  3.22791839e+000,  3.52067229e-008,
        4.58851424e+000,  3.17673388e-001,  1.05908400e+001,
        7.48023554e+000,  1.06741180e-003,  2.04901185e-001,
        3.16552976e+000,  3.14444056e+000,  4.99730330e-001,
        3.46702487e-001,  4.86842092e-020,  3.54749059e-008,
        2.72288078e-001,  2.92967640e-001,  1.80106770e+001,
        8.78002066e+000,  3.01159008e+000,  9.98320310e-011,
        3.47372677e+000,  5.72517372e-004,  6.12222778e-004,
        5.03238955e-001,  4.28690756e-020,  5.80779391e-001,
        3.10755358e-008,  5.88524285e+000,  9.24506597e+000,
        5.95090305e+000,  5.68401488e+000,  7.77445803e+000,
        4.98769973e+000,  3.13322578e+000,  3.79230211e-001,
        1.35528264e-001,  1.36116235e-001,  2.51029577e-001,
        3.69570465e-001,  9.62746865e+000,  1.85022353e+001,
        1.37574085e+001,  2.58588668e+000,  2.41177485e+000,
        5.06334133e-001,  0.00000000e+000,  6.66032337e+000,
        0.00000000e+000,  9.14736153e+000,  6.79472274e+000,
        5.62020259e+000,  7.71818311e+000,  2.06068672e+000,
        1.26262738e-001,  2.40206336e-001,  3.86810961e-001,
        2.06671831e-001,  2.61175345e-001,  1.14522608e-001,
        3.04110518e-001,  1.44757480e+001,  6.81681188e-011,
        3.90880417e-004,  6.00835814e-001,  3.91488369e-001,
        3.32154814e-020,  2.41023868e-008,  3.48185978e-002,
        5.52069742e+000,  5.79057164e+000,  4.28670136e-011,
        4.90651095e+001,  7.31295247e-017,  2.86683386e-001,
        3.11231123e-008,  1.61595020e-070,  5.23458263e-104,
        3.25908764e-065,  2.30214168e-028,  1.86625941e-001,
        1.35693615e-002,  2.75540460e-001,  5.15000845e-071,
        1.51380241e-003,  2.55975043e-001,  7.33829095e-004,
        5.27405041e-104,  3.40795200e-006,  2.35891866e-006,
        6.35427799e-070,  1.38042342e-045,  7.29195943e-001,
        1.69761890e-069,  3.77812056e-004,  1.03904615e-003,
        2.11616818e-004,  2.77885427e-004,  6.03285258e-004,
        5.59463970e-004,  9.92581409e-004,  2.58025056e-004,
        6.96629679e-039,  7.55350041e-116,  8.75811507e-051,
        2.96917064e-103,  1.03846920e-038,  5.14300860e+000,
        3.40987425e-019,  4.44318657e-003,  1.13947408e-003,
        1.03606459e-003,  9.64457348e-039,  2.97856703e-028,
        2.74405767e+000,  1.07985126e-010,  1.04374533e+001,
        3.17329206e-004,  9.87620024e+000,  9.54622314e-002,
        0.00000000e+000,  1.72845064e+001,  2.68030486e+001,
        1.12641487e+001,  3.92228817e+000,  1.36431596e+001,
        4.71710843e+000,  1.18349061e-001,  1.32279537e+000,
        1.01376731e+000,  8.84910428e-024,  4.52179724e-027,
        3.19218568e-110,  8.40414891e-018,  2.25625137e-002,
        2.92455243e-081,  2.79641264e-020,  1.21129355e-008,
        5.79889016e-007,  1.44314440e-002,  6.58265537e-024,
        5.93610104e-019,  1.71958172e-021,  5.47324857e-015,
        4.02765678e-003,  5.86555998e-006,  3.20506923e-092,
        5.37883456e-024,  0.00000000e+000,  1.87738522e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.13652040e-308,
        2.19661735e-001,  1.42812010e-001,  3.70400431e-009,
        0.00000000e+000,  2.59592678e-080,  0.00000000e+000,
        0.00000000e+000,  3.84411992e-031,  0.00000000e+000,
        0.00000000e+000,  5.85386483e-003,  1.76916681e-005,
        3.78487655e-078,  0.00000000e+000,  3.68168633e-031,
        4.70336170e-080,  1.15354274e-077,  1.03854391e-030,
        2.18478693e-017,  1.37952948e-062,  1.47122394e-006,
        1.01123964e-005,  4.94866468e-006,  4.71974673e-012,
        0.00000000e+000,  6.88711905e-028,  6.43370494e-011,
        1.17057177e-017,  3.94325919e+000,  0.00000000e+000,
        0.00000000e+000,  1.85666049e-008,  4.21419605e-005,
        2.00549668e+000,  1.97594959e-004,  3.14896060e-080,
        0.00000000e+000,  0.00000000e+000,  6.29385207e-031,
        8.06503557e-018,  4.33647088e-017,  2.15553867e-025,
        5.99916502e-028,  3.37028898e-019,  1.08605771e-027,
        8.89354880e-013,  1.63744334e-003,  1.10027936e-016,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.52003391e-027]), OrderedDict([('R2', 0.02098234772404506), ('root_mean_squared_error', 139.81061331165657), ('mean_absolute_error', 103.32189666716533), ('rmse_over_stdev', 1.4637963935658438)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_3'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
